In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import tensorflow as tf
import numpy as np
import re

In [6]:
filename = "/content/drive/My Drive/Alice2.txt"
raw_text = open(filename).read()
print(raw_text[:100])

CHAPTER I. Down the Rabbit-Hole

Alice was beginning to get very tired of sitting by her sister on t


In [0]:
# create mapping of unique words to integers, and reverse
raw_text_words=re.findall(r"[\w]+|[.!?;-]", raw_text)
#raw_text_words = raw_text.split()
#print(raw_text_words)

In [0]:
words = sorted(list(set(raw_text_words)))
#word to integer mapping
word_to_int = dict((w, i) for i, w in enumerate(words))
#integer to word mapping
int_to_word = dict((i, w) for i, w in enumerate(words))

In [0]:
#print(word_to_int)
#print(int_to_word)
#print(words)

In [10]:
n_words = len(raw_text_words)
n_vocab = len(words)
print("Total Words in the document: ", n_words)
print("Total Vocab: ", n_vocab)

Total Words in the document:  4696
Total Vocab:  1014


In [0]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 30
dataX = []
dataY = []
for i in range(0, n_words - seq_length, 1):
    seq_in = raw_text_words[i:i + seq_length]
    seq_out = raw_text_words[i + seq_length]
    dataX.append([word_to_int[word] for word in seq_in])
    dataY.append(word_to_int[seq_out])

In [12]:
dataX[:5]

[[19,
  56,
  2,
  31,
  884,
  102,
  1,
  53,
  10,
  962,
  205,
  911,
  431,
  949,
  910,
  645,
  807,
  233,
  476,
  804,
  649,
  884,
  191,
  170,
  645,
  467,
  637,
  911,
  325,
  650],
 [56,
  2,
  31,
  884,
  102,
  1,
  53,
  10,
  962,
  205,
  911,
  431,
  949,
  910,
  645,
  807,
  233,
  476,
  804,
  649,
  884,
  191,
  170,
  645,
  467,
  637,
  911,
  325,
  650,
  657],
 [2,
  31,
  884,
  102,
  1,
  53,
  10,
  962,
  205,
  911,
  431,
  949,
  910,
  645,
  807,
  233,
  476,
  804,
  649,
  884,
  191,
  170,
  645,
  467,
  637,
  911,
  325,
  650,
  657,
  933],
 [31,
  884,
  102,
  1,
  53,
  10,
  962,
  205,
  911,
  431,
  949,
  910,
  645,
  807,
  233,
  476,
  804,
  649,
  884,
  191,
  170,
  645,
  467,
  637,
  911,
  325,
  650,
  657,
  933,
  783],
 [884,
  102,
  1,
  53,
  10,
  962,
  205,
  911,
  431,
  949,
  910,
  645,
  807,
  233,
  476,
  804,
  649,
  884,
  191,
  170,
  645,
  467,
  637,
  911,
  325,
  650,
  657,


In [13]:
dataY[:5]

[657, 933, 783, 450, 677]

In [14]:
int_to_word[9]

'Ah'

In [15]:
int_to_word[30]

'Don'

In [16]:
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  4666


In [0]:
# reshape X to be [samples, time steps, features]
x_train = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
x_train = x_train / float(n_vocab)
# one hot encode the output variable
y_train = np.eye(n_vocab)[dataY]

In [18]:
print(x_train[:2])

[[[0.01873767]
  [0.05522682]
  [0.00197239]
  [0.03057199]
  [0.87179487]
  [0.10059172]
  [0.00098619]
  [0.05226824]
  [0.00986193]
  [0.94871795]
  [0.20216963]
  [0.89842209]
  [0.42504931]
  [0.93589744]
  [0.8974359 ]
  [0.63609467]
  [0.79585799]
  [0.22978304]
  [0.46942801]
  [0.79289941]
  [0.64003945]
  [0.87179487]
  [0.18836292]
  [0.16765286]
  [0.63609467]
  [0.46055227]
  [0.62820513]
  [0.89842209]
  [0.32051282]
  [0.64102564]]

 [[0.05522682]
  [0.00197239]
  [0.03057199]
  [0.87179487]
  [0.10059172]
  [0.00098619]
  [0.05226824]
  [0.00986193]
  [0.94871795]
  [0.20216963]
  [0.89842209]
  [0.42504931]
  [0.93589744]
  [0.8974359 ]
  [0.63609467]
  [0.79585799]
  [0.22978304]
  [0.46942801]
  [0.79289941]
  [0.64003945]
  [0.87179487]
  [0.18836292]
  [0.16765286]
  [0.63609467]
  [0.46055227]
  [0.62820513]
  [0.89842209]
  [0.32051282]
  [0.64102564]
  [0.64792899]]]


In [19]:
print(y_train[2])

[0. 0. 0. ... 0. 0. 0.]


In [20]:
print(x_train.shape)

(4666, 30, 1)


In [21]:
print(y_train.shape)

(4666, 1014)


In [0]:
batch_size = y_train.shape[0]

In [23]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32,  shape=(None, seq_length, 1))
Y = tf.placeholder(tf.float32,  shape=(None, n_vocab))
num_layers = 2   
hidden_units=200
#rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=hidden_units, activation=tf.nn.tanh)
layers = [tf.nn.rnn_cell.GRUCell(num_units=hidden_units, activation=tf.nn.tanh)
        for _ in range(num_layers)]

cells = tf.nn.rnn_cell.MultiRNNCell(layers)
outputs, state = tf.nn.dynamic_rnn(cells,X,dtype=tf.float32)
logits = tf.layers.dense(outputs[:,-1], n_vocab)
prob = tf.nn.softmax(logits)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
                             logits=logits, labels=Y))
train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
acc = tf.reduce_mean(tf.cast(
                        tf.equal(
                            tf.argmax(logits, 1),
                            tf.argmax(Y, 1),
                        ),
                        tf.float32,
                    ))


Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.dense instead.


In [24]:

epochs =200
batch_size=100
sess = tf.Session()
sess.run(tf.global_variables_initializer())
#t0,t1, t2=sess.run([outputs,logits, prob], feed_dict={X:x_train, Y:y_train})
for i in range(epochs):
  for j in range(0, x_train.shape[0],batch_size):
    x_batch = x_train[j:j+batch_size]
    y_batch = y_train[j:j+batch_size]
    loss_, train_, acc_ = sess.run([loss,train,acc], feed_dict={X:x_batch, Y:y_batch})
  print("loss:",loss_," Accuracy:",acc_)

loss: 6.2441244  Accuracy: 0.0
loss: 5.9167237  Accuracy: 0.07575758
loss: 5.8988695  Accuracy: 0.015151516
loss: 5.831293  Accuracy: 0.015151516
loss: 5.849607  Accuracy: 0.015151516
loss: 5.844077  Accuracy: 0.015151516
loss: 5.810595  Accuracy: 0.015151516
loss: 5.816204  Accuracy: 0.015151516
loss: 5.8134785  Accuracy: 0.015151516
loss: 5.813467  Accuracy: 0.015151516
loss: 5.8017116  Accuracy: 0.015151516
loss: 5.8185725  Accuracy: 0.015151516
loss: 5.826115  Accuracy: 0.015151516
loss: 5.823043  Accuracy: 0.015151516
loss: 5.826364  Accuracy: 0.015151516
loss: 5.8020253  Accuracy: 0.015151516
loss: 5.828991  Accuracy: 0.015151516
loss: 5.80767  Accuracy: 0.015151516
loss: 5.8119855  Accuracy: 0.015151516
loss: 5.801748  Accuracy: 0.09090909
loss: 5.7487383  Accuracy: 0.10606061
loss: 5.828947  Accuracy: 0.060606062
loss: 5.813122  Accuracy: 0.015151516
loss: 5.760707  Accuracy: 0.045454547
loss: 5.7102284  Accuracy: 0.07575758
loss: 5.877468  Accuracy: 0.015151516
loss: 5.7480917

In [25]:
print(outputs)

Tensor("rnn/transpose_1:0", shape=(?, 30, 200), dtype=float32)


In [0]:
# # pick a random seed
# start = np.random.randint(0, len(dataX)-1)
# #start=0
# pattern = dataX[start]

In [0]:
def encode_test(txt):
  raw_text_words=re.findall(r"[\w]+|[.!?;-]", txt)
  if len(raw_text_words) != seq_length:
     raw_text_words = ["!"]*(seq_length-len(raw_text_words))+raw_text_words
  return [word_to_int[word] for word in raw_text_words]
  

In [28]:
test="The rabbit was sitting by "
pattern = encode_test(test)
print(pattern)
print(len(pattern))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 119, 719, 962, 807, 233]
30


In [29]:
print("Seed:")
print (' '.join([int_to_word[value] for value in pattern]))

Seed:
! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! The rabbit was sitting by


In [0]:
# generate text
out=" "
for i in range(100):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = sess.run(prob,feed_dict={X:x})
    #print(prob.shape)
    index = np.argmax(prediction)
    #print(index)
    result = int_to_word[index]
    seq_in = [int_to_word[value] for value in pattern]
    out=out+ result+ " "
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

In [33]:
print(out)

 to to to to she she she she her . and when swimming back she she she to and she she to to to to to to to to to to she she she she she her . Oh WILL worse ! on the to she . I the - - she to to to to to to to to to she she of her . I when very a I smiling a she a I and have I while to to to to to to to to to to she of her the the - she the the and 
